In [ ]:
import pandas as pd
import numpy as np
from pandas.tseries.offsets import BDay
import holidays
import random

In [ ]:
start_year = 2024
end_year = 2025

# Define the date range
start_date = f'{start_year}-09-01'
end_date = f'{end_year}-04-30'
date_range = pd.date_range(start=start_date, end=end_date, freq='B')

# Define your custom holidays
custom_holidays = holidays.Canada(years=[start_year, end_year])

# Add UofT reading weeks and Christmas break to custom holidays
uoft_days_off = [
    pd.date_range(start='2024-10-28', end='2024-11-01'), #Fall reading week
    pd.date_range(start='2025-02-17', end='2025-02-21'), #Winter reading week
    pd.date_range(start='2024-12-24', end='2025-01-05') #Christmas break
]

# Add Uoft Exam periods
exam_periods =[
    pd.date_range(start='2024-12-5', end='2024-12-23'),
    pd.date_range(start='2025-04-08', end='2025-04-30')
]

#Exam dates (days during exam period where i need to go)
exam_dates = [date for period in exam_periods for date in random.sample(list(period), 5)]

In [ ]:
exam_dates

[Timestamp('2024-12-23 00:00:00'),
 Timestamp('2024-12-16 00:00:00'),
 Timestamp('2024-12-10 00:00:00'),
 Timestamp('2024-12-18 00:00:00'),
 Timestamp('2024-12-05 00:00:00'),
 Timestamp('2025-04-10 00:00:00'),
 Timestamp('2025-04-19 00:00:00'),
 Timestamp('2025-04-14 00:00:00'),
 Timestamp('2025-04-12 00:00:00'),
 Timestamp('2025-04-13 00:00:00')]

In [ ]:
for period in exam_periods:
    for day in period:
        custom_holidays.update({day.date(): "Exam Study"})


# Flatten the list of reading week dates and add to custom holidays
for week in uoft_days_off:
    for day in week: # Iterate over individual dates in each week
        custom_holidays.update({day.date(): "Uoft Holiday"}) # Add each date to custom holidays

# Filter out holidays
business_days = date_range[~date_range.isin(custom_holidays)]

#Add exam dates to buisness days
business_days = business_days.append(pd.DatetimeIndex(exam_dates))


# Count weekdays per month
weekdays_per_month = business_days.to_series().groupby(business_days.to_period('M')).count()

# Create a DataFrame
df = pd.DataFrame(weekdays_per_month, columns=['Weekdays'])
df.index.name = 'Month'
df.reset_index(inplace=True)
df['Month'] = df['Month'].dt.strftime('%B')
df

,Month,Weekdays
0,September,20
1,October,19
2,November,20
3,December,8
4,January,20
5,February,15
6,March,21
7,April,10


In [ ]:
standard = 8.85
first_30_trips_fare = round(standard*0.6, 2) #aka presto fare
_31_to_40_trips_fare = round(standard*0.05, 2)
more_thab_40_trips_fare = 0
trips_per_day = 2

In [ ]:
def calculate_fare(days: int) -> float:

    if not isinstance(days, int) or days <= 0 :
        raise Exception("Sorry, only integers and only numbers above zero")
        return 0

    trips = days * trips_per_day

    if trips <= 30:
        return trips * first_30_trips_fare

    elif trips <= 40:
        return (30 * first_30_trips_fare) + ((trips - 30) * _31_to_40_trips_fare)

    else:
        return (30 * first_30_trips_fare) + ((40 - 30) * _31_to_40_trips_fare) + ((trips - 40) * more_thab_40_trips_fare)

    return 0

In [ ]:
df['Fare'] = df['Weekdays'].apply(calculate_fare)

In [ ]:
df

,Month,Weekdays,Fare
0,September,20,163.70
1,October,19,162.82
2,November,20,163.70
3,December,8,84.96
4,January,20,163.70
5,February,15,159.30
6,March,21,163.70
7,April,10,106.20
